In [8]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import cv2

In [9]:
nx = 400 # number of grid points in the horizontal direction
nz = 400 # number of grid points in the vertical direction
dd = 5.0 # grid cell size
nt = 1000 # number of time samples to be modeled
dt = 0.0005 # time step
srcx = 900 # source horizontal location in meters
srcz = 800 # source vertical location in meters

nabs = 40 # number of absorbing cells on the boundary
a = 0.0053 # strength of sponge layer
FreeSurf=False # free surface condition of top (False for now)

vp = np.arange(2000,4500,(4500-2000)/nz)
vp = np.tile(np.expand_dims(vp,0),[nx,1]) # just coming up with a velocity model

#vp = np.ones((nx,nz))*3000.0

time = np.arange(0,nt*dt,dt) # time vector
f0 = 15. # central frequency of the wavelet
t0 = 1/f0 # a shift to make sure wavelet is causal
wav  = (1.0-2.0*np.power(np.pi*f0*(time-t0),2))*np.exp(-np.power(np.pi*f0*(time-t0),2)) # computing the wavelet

In [10]:
# def make_input(vp, wave, time, srcx, srcz, dd):
#     dd = int(dd)
#     srcx = srcx // dd
#     srcz = srcz // dd
#     wave += 1
#     out_channels = len(time)
#     vp_out = np.zeros((out_channels, vp.shape[0], vp.shape[1]))
#     for i in range(out_channels):
#         vp_temp = np.copy(vp)
#         vp_temp[srcx, srcz] *= wave[i]
#         vp_out[i] = vp_temp
#     return vp_out

In [11]:
def make_input(vp, wave, time, srcx, srcz, dd):
    dd = int(dd)
    srcx = srcx // dd
    srcz = srcz // dd
    wave += 1
    vp_out = np.copy(vp)
    vp_out[srcx, srcz] *= wave.max()
    return vp_out

In [12]:
inp = make_input(vp, wav, time, srcx, srcz, dd)

In [13]:
inp.shape

(400, 400)

In [20]:
# class Generator(nn.Module):
#     '''
#     Generator Class
#     Values:
#         z_dim: the dimension of the noise vector, a scalar
#         im_chan: 
#         hidden_dim: the inner dimension, a scalar
#     '''
#     def __init__(self, z_dim=400*400, im_chan=1000, hidden_dim=32000):
#         super(Generator, self).__init__()
#         self.z_dim = z_dim
#         # Build the neural network
#         self.gen = nn.Sequential(
#             self.make_gen_block(z_dim, hidden_dim * 4, kernel_size=4), #4x4
#             self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size=2, stride=3), #11x11
#             self.make_gen_block(hidden_dim * 2, hidden_dim, kernel_size=3, stride=3), #33x33
#             self.make_gen_block(hidden_dim, hidden_dim // 2, kernel_size=4, stride=3), #100x100
#             self.make_gen_block(hidden_dim // 2, hidden_dim // 4, kernel_size=4, stride=2, padding=1), #200x200
#             self.make_gen_block(hidden_dim // 4, im_chan, kernel_size=2, stride=2, final_layer=True) #400x400
#         )

#     def make_gen_block(self, input_channels, output_channels, kernel_size=3, 
#                        stride=2, padding=0, final_layer=False):
#         '''
#         Function to return a sequence of operations corresponding to a generator block of DCGAN;
#         a transposed convolution, a batchnorm (except in the final layer), and an activation.
#         Parameters:
#             input_channels: how many channels the input feature representation has
#             output_channels: how many channels the output feature representation should have
#             kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
#             stride: the stride of the convolution
#             padding: the padding of the convolution
#             final_layer: a boolean, true if it is the final layer and false otherwise 
#                       (affects activation and batchnorm)
#         '''
#         if not final_layer:
#             return nn.Sequential(
#                 nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride, padding),
#                 nn.BatchNorm2d(output_channels),
#                 nn.ReLU(inplace=True),
#             )
#         else:
#             return nn.Sequential(
#                 nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride, padding),
#                 nn.Tanh(),
#             )

#     def forward(self, noise):
#         '''
#         Function for completing a forward pass of the generator: Given a noise tensor,
#         returns generated images.
#         Parameters:
#             noise: a noise tensor with dimensions (n_samples, z_dim)
#         '''
# #         x = noise.view(len(noise), self.z_dim, 1, 1)
#         x = noise.view(self.z_dim, 1, 1)
#         return self.gen(x)

# def get_noise(n_samples, z_dim, device='cpu'):
#     '''
#     Function for creating noise vectors: Given the dimensions (n_samples, z_dim)
#     creates a tensor of that shape filled with random numbers from the normal distribution.
#     Parameters:
#       n_samples: the number of samples to generate, a scalar
#       z_dim: the dimension of the noise vector, a scalar
#       device: the device type
#     '''
#     noise = torch.FloatTensor(make_input(vp, wav, time, srcx, srcz, dd))
#     return noise.to(device)

In [ ]:
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_chan: 
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, z_dim=1, im_chan=1000, hidden_dim=32000):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        # Build the neural network
        self.gen = nn.Sequential(
            self.make_gen_block(z_dim, hidden_dim * 4, kernel_size=4), #4x4
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size=2, stride=3), #11x11
            self.make_gen_block(hidden_dim * 2, hidden_dim, kernel_size=3, stride=3), #33x33
            self.make_gen_block(hidden_dim, hidden_dim // 2, kernel_size=4, stride=3), #100x100
            self.make_gen_block(hidden_dim // 2, hidden_dim // 4, kernel_size=4, stride=2, padding=1), #200x200
            self.make_gen_block(hidden_dim // 4, im_chan, kernel_size=2, stride=2, final_layer=True) #400x400
        )

    def make_gen_block(self, input_channels, output_channels, kernel_size=3, 
                       stride=2, padding=0, final_layer=False):
        '''
        Function to return a sequence of operations corresponding to a generator block of DCGAN;
        a transposed convolution, a batchnorm (except in the final layer), and an activation.
        Parameters:
            input_channels: how many channels the input feature representation has
            output_channels: how many channels the output feature representation should have
            kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
            stride: the stride of the convolution
            padding: the padding of the convolution
            final_layer: a boolean, true if it is the final layer and false otherwise 
                      (affects activation and batchnorm)
        '''
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride, padding),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride, padding),
                nn.Tanh(),
            )

    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor,
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
#         x = noise.view(len(noise), self.z_dim, 1, 1)
        x = noise
        return self.gen(x)

def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim)
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
      n_samples: the number of samples to generate, a scalar
      z_dim: the dimension of the noise vector, a scalar
      device: the device type
    '''
    noise = torch.FloatTensor(make_input(vp, wav, time, srcx, srcz, dd))
    return noise.to(device)